# Pipeline example

In [1]:
import pandas as pd
import geopandas as gpd

blocks = pd.read_pickle('/root/blocksnet/tests/data/blocks.pickle')
blocks_gdf = gpd.GeoDataFrame(blocks)

Initialize graph

In [2]:
from blocksnet.relations import generate_adjacency_graph

adjacency_graph = generate_adjacency_graph(blocks_gdf, 10)

2025-05-29 16:10:47.517 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_nodes:10 - Generating nodes
2025-05-29 16:10:47.520 | INFO     | blocksnet.relations.adjacency.core:_generate_adjacency_edges:15 - Generating edges
2025-05-29 16:10:48.014 | SUCCESS  | blocksnet.relations.adjacency.core:generate_adjacency_graph:38 - Adjacency graph successfully generated: 2700 nodes, 8966 edges


## Density

In [3]:
from blocksnet.machine_learning.regression import DensityRegressor

dr = DensityRegressor()

/root/blocksnet/.venv/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/root/blocksnet/.venv/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


### Predicting fsi, gsi, mxi

In [4]:
result_df = dr.evaluate(blocks_gdf, adjacency_graph)

2025-05-29 16:10:50.267 | WARNING  | blocksnet.machine_learning.regression.density.schemas:_before_validate:33 - Not valid format. Trying to one hot from land_use column
2025-05-29 16:10:50.519 | INFO     | blocksnet.preprocessing.feature_engineering.core:_calculate_usual_features:35 - Calculating usual features


In [5]:
result_df

,fsi,gsi,mxi
0,0.047580,0.031040,0.597212
1,0.185372,0.087705,0.714367
2,0.154388,0.096295,0.062276
3,0.411970,0.141523,0.392915
4,0.079224,0.019165,0.141795
...,...,...,...
2695,0.142503,0.082639,0.212326
2696,0.175975,0.094356,0.193998
2697,0.172016,0.092913,0.197617
2698,0.252905,0.081366,0.597263


In [6]:
blocks_gdf['site_area'] = blocks_gdf.geometry.area
result_df = result_df.join(blocks_gdf[['site_area']])

In [7]:
result_df

,fsi,gsi,mxi,site_area
0,0.047580,0.031040,0.597212,202399.732194
1,0.185372,0.087705,0.714367,201529.493481
2,0.154388,0.096295,0.062276,164663.859829
3,0.411970,0.141523,0.392915,161571.832511
4,0.079224,0.019165,0.141795,16.489152
...,...,...,...,...
2695,0.142503,0.082639,0.212326,22875.011366
2696,0.175975,0.094356,0.193998,1214.910251
2697,0.172016,0.092913,0.197617,2449.592208
2698,0.252905,0.081366,0.597263,5.073803


# Values below zero

In [8]:
(result_df[[
    'site_area',
    'fsi',
    'gsi',
    'mxi',
]] < 0).sum()

site_area     0
fsi          42
gsi           7
mxi          43
dtype: int64

In [9]:
result_df[result_df[[
    'site_area',
    'fsi',
    'gsi',
    'mxi',
]] < 0] = 0

# Calculating dev indicators from area, fsi, gsi, mxi

In [10]:
from blocksnet.analysis.indicators import calculate_development_indicators 

blocks = calculate_development_indicators(result_df[[
    'site_area',
    'fsi',
    'gsi',
    'mxi',
]])

# Adding population value

In [11]:
blocks['population'] = blocks_gdf['population']
blocks['buildings_count'] = blocks_gdf['objects_count']
blocks['is_living'] = blocks_gdf['is_living']

mask = blocks['population'] == 0
blocks.loc[mask, 'population'] = blocks.loc[mask, 'living_area'] / 20

/tmp/ipykernel_312464/2758934810.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.87560915e+02 1.33436088e+03 7.91590142e+01 ... 4.16349704e+00
 3.83201793e-02 5.67298477e-04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  blocks.loc[mask, 'population'] = blocks.loc[mask, 'living_area'] / 20


In [12]:
blocks['land_use'] = blocks_gdf['land_use']
blocks['land_use'] = blocks['land_use'].apply(lambda lu : lu.value if lu is not None else 'unknown')

land_uses = blocks.groupby('land_use')['site_area'].sum().to_frame().T.reset_index()

# Adding landuse info

# Predicting social indicators

In [13]:
mo_df = blocks.sum().to_frame().T

In [14]:
mo_df = pd.concat([mo_df, land_uses], axis=1)

In [15]:
mo_df

,site_area,fsi,gsi,mxi,build_floor_area,footprint_area,living_area,non_living_area,population,buildings_count,...,land_use,index,agriculture,business,industrial,recreation,residential,special,transport,unknown
0,590964488.275079,464.037196,204.326412,910.797052,66766159.358916,23705851.222585,26838110.516335,39928048.842581,533461.62413,1768,...,agriculturebusinesstransportresidentialunknown...,site_area,799097.60226,1.295449e+06,1.410717e+06,3.049707e+06,1.170972e+07,1.526682e+06,237916.846397,5.709352e+08


In [16]:
mo_df['latitude'] = blocks_gdf.geometry.union_all().centroid.x
mo_df['longitude'] = blocks_gdf.geometry.union_all().centroid.y

In [17]:
from blocksnet.machine_learning.regression import SocialRegressor

sr = SocialRegressor()

In [18]:
y_pred, pi_lower, pi_upper = sr.evaluate(mo_df)

In [19]:
y_pred

,nursing_home_count,hotel_count,theatre_count,cinema_count,secondary_vocational_education_institutions_count,university_count,stadium_count,emergency_medical_service_stations_count,kindergarten_count,hostel_count,...,post_count,swimming_pool_count,library_count,guest_house_count,fire_safety_facilities_count,restaurant_count,police_count,museum_count,bank_count,pitch_count
0,1.724709,38.906454,5.669712,1.192193,8.415945,11.056443,10.359553,2.869026,87.163117,34.825633,...,2.820628,25.45424,1.062628,2.423518,33.45202,37.470833,8.261214,6.859682,3.886751,267.922235
